### Bloc 3.12 – Méthode M3 : analyse spectrale (slope β) sur le sous‑échantillon (S0)

Objectif : ajouter une **troisième famille d’indicateur de complexité**, basée sur le **spectre
de puissance** de chaque fenêtre (série 1D), et la comparer à M1/M2.

Principe :

- Pour chaque fenêtre (W, G) de S0 :
  1. on prend la série mensuelle S0 sur l’intervalle `[start_index, end_index]` (longueur W) ;
  2. on calcule le **spectre de puissance** via la méthode de Welch (`scipy.signal.welch`) ;
  3. on ajuste une droite sur la relation  
     \[
     \log_{10} P(f) \approx \alpha + \beta \log_{10} f
     \]
     sur une **bande fréquentielle intermédiaire** (on exclut les toutes premières et dernières
     fréquences pour éviter les effets de bord) ;
  4. on retient :
     - la pente `β` (spectral slope),
     - l’ordonnée à l’origine `α`,
     - un coefficient de détermination `R²` pour la qualité du fit.

Remarques importantes :

- Ici, `β` est utilisé comme **indice spectral** (plus négatif → spectre plus “rouge” → structure
  plus longue‑mémoire).  
- Nous **ne traduisons pas directement β en “dimension d”** (ce serait possible sous des hypothèses
  fortes sur le processus, mais ce serait trop spéculatif).  
- M3 servira donc surtout à **contrôler la cohérence** avec M1/M2 et à repérer des régimes
  spectraux différents (par exemple selon W).

Sorties :

- Un fichier par fenêtre, avec un enregistrement par fenêtre de calibration :

  - `data_phase2/d_estimates_calibration/M3_S0_spectral_per_window.csv`  

  colonnes : `series`, `window_id`, `window_size_months`, `stride_months`,
  `start_date`, `end_date`, `n_points`, `beta`, `alpha`, `r2`, `f_low_used`,
  `f_high_used`, `n_freq_used`.

- Un résumé par (W, G) :

  - `data_phase2/d_estimates_calibration/M3_S0_spectral_summary.csv`  

  colonnes : `series`, `window_size_months`, `stride_months`,
  `beta_mean`, `beta_std`, `beta_min`, `beta_max`, `r2_mean`, `n_windows`.

Nous utilisons la même série S0 et le même sous‑échantillon de calibration que pour M1/M2.

In [23]:
# Bloc 3.12 – Analyse spectrale M3 (slope β) sur S0 – Sous-échantillon de calibration

import numpy as np
import pandas as pd
from pathlib import Path
from scipy.signal import welch

# 3.12.1 – Vérifier df_ts (S0) et le sous-échantillon de fenêtres

try:
    df_ts
except NameError:
    raise RuntimeError(
        "df_ts n'est pas défini. Assurez-vous d'avoir exécuté le Bloc 3.0 avant ce bloc."
    )

value_col = "Monthly Mean Total Sunspot Number"

WINDOWS_DIR = PHASE2_ROOT / "data_phase2" / "windows"
WINDOW_CALIB_PATH = WINDOWS_DIR / "window_calibration_subset.csv"

if not WINDOW_CALIB_PATH.exists():
    raise FileNotFoundError(
        f"window_calibration_subset.csv introuvable à : {WINDOW_CALIB_PATH}.\n"
        "Assurez-vous d'avoir exécuté le Bloc 3.1."
    )

df_windows_calib = pd.read_csv(
    WINDOW_CALIB_PATH,
    parse_dates=["start_date", "end_date"],
)

print(f"Sous-échantillon de calibration (M3) chargé : {len(df_windows_calib)} fenêtres.")
display(df_windows_calib.head())

# 3.12.2 – Paramètres pour le calcul spectral

FS = 1.0  # fréquence d'échantillonnage : 1 point par mois
FREQ_Q_LOW = 0.1   # on garde les fréquences entre les quantiles 10% et 90%
FREQ_Q_HIGH = 0.9

print(f"\nParamètres spectre : fs={FS}, quantiles de fréquence [{FREQ_Q_LOW}, {FREQ_Q_HIGH}]")

# 3.12.3 – Fonction utilitaire pour la pente spectrale sur une fenêtre

def spectral_slope_beta(series_values, fs=1.0, q_low=0.1, q_high=0.9):
    """
    Calcule la pente β de log10 P(f) ~ α + β log10 f sur une bande intermédiaire.
    Retourne (beta, alpha, r2, f_low_used, f_high_used, n_freq_used)
    """
    # Welch PSD
    freqs, psd = welch(series_values, fs=fs, nperseg=len(series_values))

    # On ignore la fréquence 0 et les puissances nulles
    mask = (freqs > 0) & (psd > 0)
    freqs = freqs[mask]
    psd = psd[mask]

    if len(freqs) < 5:
        return np.nan, np.nan, np.nan, np.nan, np.nan, 0

    # Bande intermédiaire via quantiles sur les fréquences
    f_low = np.quantile(freqs, q_low)
    f_high = np.quantile(freqs, q_high)
    band_mask = (freqs >= f_low) & (freqs <= f_high)

    freqs_band = freqs[band_mask]
    psd_band = psd[band_mask]

    if len(freqs_band) < 5:
        # si bande trop étroite, on élargit à toutes les fréquences >0
        freqs_band = freqs
        psd_band = psd
        f_low, f_high = freqs_band.min(), freqs_band.max()

    x = np.log10(freqs_band)
    y = np.log10(psd_band)

    # Ajustement linéaire
    coeffs = np.polyfit(x, y, 1)
    beta = coeffs[0]
    alpha = coeffs[1]

    # Coefficient de détermination R^2
    y_pred = np.polyval(coeffs, x)
    ss_res = np.sum((y - y_pred) ** 2)
    ss_tot = np.sum((y - y.mean()) ** 2)
    r2 = 1.0 - ss_res / ss_tot if ss_tot > 0 else np.nan

    return float(beta), float(alpha), float(r2), float(f_low), float(f_high), int(len(freqs_band))

# 3.12.4 – Calcul de β pour chaque fenêtre du sous-échantillon

records_per_window = []

groups = df_windows_calib.groupby(["window_size_months", "stride_months"], sort=True)

for (W, G), group in groups:
    print(f"\nTraitement M3 pour (W={W}, G={G}) – {len(group)} fenêtres")

    for _, row in group.iterrows():
        window_id = int(row["window_id"])
        start_idx = int(row["start_index"])
        end_idx = int(row["end_index"])
        start_date = row["start_date"]
        end_date = row["end_date"]
        n_points = int(row["n_points"])

        values = df_ts.iloc[start_idx : end_idx + 1][value_col].to_numpy(dtype=float)

        if len(values) != n_points or len(values) != W:
            raise RuntimeError(
                f"Longueur inattendue pour window_id={window_id} (attendu W={W}, n_points={n_points}, "
                f"obtenu {len(values)})."
            )

        beta, alpha, r2, f_low, f_high, n_freq_used = spectral_slope_beta(
            values, fs=FS, q_low=FREQ_Q_LOW, q_high=FREQ_Q_HIGH
        )

        records_per_window.append(
            {
                "series": "S0",
                "window_id": window_id,
                "window_size_months": int(W),
                "stride_months": int(G),
                "start_date": start_date,
                "end_date": end_date,
                "n_points": int(n_points),
                "beta": beta,
                "alpha": alpha,
                "r2": r2,
                "f_low_used": f_low,
                "f_high_used": f_high,
                "n_freq_used": n_freq_used,
            }
        )

# 3.12.5 – DataFrame par fenêtre et résumé par (W,G)

df_M3_per_window = pd.DataFrame.from_records(records_per_window)

print("\nAperçu M3 par fenêtre (S0, calibration) :")
display(df_M3_per_window.head())

summary_records = []

for (W, G), grp in df_M3_per_window.groupby(["window_size_months", "stride_months"], sort=True):
    betas = grp["beta"].to_numpy(dtype=float)
    r2s = grp["r2"].to_numpy(dtype=float)

    summary_records.append(
        {
            "series": "S0",
            "window_size_months": int(W),
            "stride_months": int(G),
            "n_windows": int(len(grp)),
            "beta_mean": float(np.nanmean(betas)),
            "beta_std": float(np.nanstd(betas)),
            "beta_min": float(np.nanmin(betas)),
            "beta_max": float(np.nanmax(betas)),
            "r2_mean": float(np.nanmean(r2s)),
        }
    )

df_M3_summary = pd.DataFrame.from_records(summary_records)

print("\nRésumé M3 (S0, par combinaison W,G) :")
display(df_M3_summary)

# 3.12.6 – Sauvegarde des résultats

DEST_DIR = PHASE2_ROOT / "data_phase2" / "d_estimates_calibration"
DEST_DIR.mkdir(parents=True, exist_ok=True)

M3_PER_WINDOW_PATH = DEST_DIR / "M3_S0_spectral_per_window.csv"
M3_SUMMARY_PATH = DEST_DIR / "M3_S0_spectral_summary.csv"

df_M3_per_window.to_csv(M3_PER_WINDOW_PATH, index=False)
df_M3_summary.to_csv(M3_SUMMARY_PATH, index=False)

print("\nFichiers de résultats M3 (S0, calibration) sauvegardés :")
print(f"  - {M3_PER_WINDOW_PATH}")
print(f"  - {M3_SUMMARY_PATH}")

# 3.12.7 – Logging

log_message(
    "INFO",
    (
        "M3 (spectral slope β) appliqué à S0 sur le sous-échantillon de calibration "
        f"pour {len(df_M3_summary)} combinaisons (W,G). Résultats sauvegardés dans "
        f"{M3_PER_WINDOW_PATH.name} et {M3_SUMMARY_PATH.name}."
    ),
    block="BLOC_3.12",
)
log_metric(
    "M3_S0_calibration_combos",
    int(len(df_M3_summary)),
    extra={
        "per_window_path": str(M3_PER_WINDOW_PATH),
        "summary_path": str(M3_SUMMARY_PATH),
    },
)

Sous-échantillon de calibration (M3) chargé : 360 fenêtres.


,window_id,window_size_months,stride_months,start_index,end_index,start_date,end_date,n_points
0,0,60,1,0,59,1749-01-31,1753-12-31,60
1,82,60,1,82,141,1755-11-30,1760-10-31,60
2,164,60,1,164,223,1762-09-30,1767-08-31,60
3,246,60,1,246,305,1769-07-31,1774-06-30,60
4,328,60,1,328,387,1776-05-31,1781-04-30,60



Paramètres spectre : fs=1.0, quantiles de fréquence [0.1, 0.9]

Traitement M3 pour (W=60, G=1) – 40 fenêtres

Traitement M3 pour (W=60, G=6) – 40 fenêtres

Traitement M3 pour (W=60, G=12) – 40 fenêtres

Traitement M3 pour (W=132, G=1) – 40 fenêtres

Traitement M3 pour (W=132, G=6) – 40 fenêtres

Traitement M3 pour (W=132, G=12) – 40 fenêtres

Traitement M3 pour (W=264, G=1) – 40 fenêtres

Traitement M3 pour (W=264, G=6) – 40 fenêtres

Traitement M3 pour (W=264, G=12) – 40 fenêtres

Aperçu M3 par fenêtre (S0, calibration) :


,series,window_id,window_size_months,stride_months,start_date,end_date,n_points,beta,alpha,r2,f_low_used,f_high_used,n_freq_used
0,S0,0,60,1,1749-01-31,1753-12-31,60,-0.974262,2.231586,0.197296,0.065,0.451667,24
1,S0,82,60,1,1755-11-30,1760-10-31,60,1.115223,3.343228,0.270312,0.065,0.451667,24
2,S0,164,60,1,1762-09-30,1767-08-31,60,-0.675408,1.692639,0.074954,0.065,0.451667,24
3,S0,246,60,1,1769-07-31,1774-06-30,60,-0.660320,3.075125,0.145290,0.065,0.451667,24
4,S0,328,60,1,1776-05-31,1781-04-30,60,-0.263156,3.137050,0.012405,0.065,0.451667,24



Résumé M3 (S0, par combinaison W,G) :


,series,window_size_months,stride_months,n_windows,beta_mean,beta_std,beta_min,beta_max,r2_mean
0,S0,60,1,40,-0.820512,0.794362,-2.527688,1.115223,0.180644
1,S0,60,6,40,-0.804998,0.772356,-2.400873,1.310190,0.182117
2,S0,60,12,40,-0.841855,0.731168,-2.400873,1.508805,0.186580
3,S0,132,1,40,-0.829343,0.482573,-1.764759,-0.061051,0.151194
4,S0,132,6,40,-0.838808,0.474008,-1.740194,-0.047775,0.151052
5,S0,132,12,40,-0.805124,0.475225,-1.715469,0.066772,0.146791
6,S0,264,1,40,-0.796448,0.379783,-1.485000,-0.062033,0.132426
7,S0,264,6,40,-0.794694,0.380453,-1.517209,-0.062033,0.130926
8,S0,264,12,40,-0.789118,0.375630,-1.517209,-0.062033,0.128232



Fichiers de résultats M3 (S0, calibration) sauvegardés :
  - C:\Users\zackd\OneDrive\Desktop\Phase2_Tlog_v0.5\SunspotPhase2Tlog\data_phase2\d_estimates_calibration\M3_S0_spectral_per_window.csv
  - C:\Users\zackd\OneDrive\Desktop\Phase2_Tlog_v0.5\SunspotPhase2Tlog\data_phase2\d_estimates_calibration\M3_S0_spectral_summary.csv
[STEP=21][INFO][BLOC_3.12] M3 (spectral slope β) appliqué à S0 sur le sous-échantillon de calibration pour 9 combinaisons (W,G). Résultats sauvegardés dans M3_S0_spectral_per_window.csv et M3_S0_spectral_summary.csv.
[METRIC][M3_S0_calibration_combos] = 9 (step=21)


### Bloc 3.13 – Synthèse M3 (spectral slope β) sur S0 – Sous‑échantillon

Nous avons appliqué l’analyse spectrale M3 sur le même sous‑échantillon que M1/M2
(360 fenêtres S0). Pour chaque fenêtre :

- calcul du spectre de puissance via Welch (fs = 1/mois) ;
- ajustement d’une droite sur `log10 P(f)` vs `log10 f` sur une bande intermédiaire
  de fréquences (quantiles 10 %–90 %) ;
- extraction de la pente `β`, de l’ordonnée `α` et du `R²`.

Résumé par combinaison `(W, G)` (à partir de `M3_S0_spectral_summary.csv`) :

- **W = 60 mois**  
  - `beta_mean` ≈ −0.82 (selon G)  
  - `beta_std` ≈ 0.73–0.79  
  - `beta_min` ≈ −2.53, `beta_max` ≈ +1.51  
  - `r2_mean` ≈ 0.18

- **W = 132 mois**  
  - `beta_mean` ≈ −0.83  
  - `beta_std` ≈ 0.47–0.48  
  - `beta_min` ≈ −1.76…−1.72, `beta_max` ≈ −0.05…+0.07  
  - `r2_mean` ≈ 0.15

- **W = 264 mois**  
  - `beta_mean` ≈ −0.79  
  - `beta_std` ≈ 0.38  
  - `beta_min` ≈ −1.52…−1.49, `beta_max` ≈ −0.06  
  - `r2_mean` ≈ 0.13

#### Interprétation

1. **Stabilité de β par rapport à W et G**

   - Les `beta_mean` sont tous autour de **−0.8**, avec peu de variation entre W=60,132,264
     et entre G=1,6,12.  
   - M3 montre donc un **comportement spectral assez stable** : le signal ressemble à un
     processus "rouge" (~1/f^0.8) sur toutes ces échelles.

2. **Dispersion et qualité de fit**

   - Les `beta_std` sont modérés (≈0.4–0.8), `beta_min` peut descendre vers −2.5 et
     `beta_max` remonter vers ≈+1.5 → certaines fenêtres ont des spectres atypiques.  
   - Les `r2_mean` (~0.13–0.18) indiquent que le modèle "droite parfaite" sur log‑log
     n’explique pas toute la structure : le spectre n’est pas un pur power‑law,
     mais la tendance globale est bien négative.

3. **Comparaison avec M1/M2**

   - M1 & M2 montrent une **variation claire de `d` avec W** (≈3 → ≈4 → ≈5).  
   - M3 montre au contraire un **β moyen assez constant** autour de −0.8, quel que soit W.  
   → Interprétation : la "couleur" du bruit (mémoire longue) reste similaire à ces échelles,
     mais la complexité géométrique (M1/M2) augmente avec la longueur de fenêtre.

En résumé :

- M3 ne donne pas une "dimension d" directe, mais confirme un comportement spectral
  relativement homogène (β ~ −0.8) sur toutes les échelles testées.
- M1/M2 restent donc les principaux indicateurs pour la variation de `d` avec W,
  tandis que M3 joue un rôle de **contrôle spectral** (le signal garde une structure
  1/f^β similaire quelle que soit la fenêtre).

### Bloc 3.x – Position méthodologique sur `d̂` et la famille M1–M3

Dans la Phase 2, nous considérons **`d̂` comme une *distribution*** et non comme une
valeur unique “vraie” :

- chaque méthode `M_i` (M1 Levina–Bickel, M2 PR/PCA, M3 spectrale, etc.) fournit
  une **famille d’estimateurs** de la complexité ;
- `d̂` dépend des **choix méthodologiques** :
  - taille de fenêtre `W`,
  - pas de glissement `G`,
  - plage de `k` (pour M1),
  - type de normalisation (S0 vs S1),
  - méthode utilisée (M1 vs M2 vs M3).

Notre stratégie Phase 2 :

- ne pas chercher un **unique “d exact”**, mais :
  - estimer **des plages plausibles de `d`** par méthode et par `(W, G)`,
  - comparer les méthodes entre elles (M1 vs M2, M3 comme contrôle spectral),
  - propager cette **incertitude sur `d`** vers `T_log(n, d)` (par ex. via `d_min`,
    `d_mean`, `d_max`).

Ainsi, l’équation `T_log(n, d) = (d - 4) log n` reste le **cadre théorique**,
tandis que la Phase 2 se concentre sur :

- la *mesure* réaliste de `d̂` (avec plusieurs M_i),
- la documentation explicite des choix méthodo,
- et l’analyse de la robustesse des conclusions face à ces choix.

### Bloc 3.14 – M1 (Levina–Bickel) sur *toutes* les fenêtres S0

Après la phase de **calibration** sur 360 fenêtres (sous-échantillon), avec :

- choix de la plage complète `k ∈ [5, 20]` ;
- définition d’une plage **centrale** `k ∈ [10, 18]` ;
- validation de la robustesse à la normalisation (S0 vs S1) et à G,

nous appliquons maintenant M1 à **toutes les 11 682 fenêtres S0** décrites dans
[data_phase2/windows/window_definitions.csv](cci:7://file:///c:/Users/zackd/OneDrive/Desktop/Phase2_Tlog_v0.5/SunspotPhase2Tlog/data_phase2/windows/window_definitions.csv:0:0-0:0).

Pour chaque combinaison `(W, G)` :

1. nous construisons la matrice `X_all` de taille `(n_windows, W)` à partir de `df_ts` (S0) ;
2. nous calculons `d_hat(k)` pour `k = K_MIN..K_MAX` avec la fonction Levina–Bickel
   (comme en calibration) ;
3. nous résumons pour cette combinaison :
   - sur la plage complète de `k` (diagnostic global) ;
   - sur la plage **centrale** `k ∈ [K_CORE_MIN, K_CORE_MAX]` (estimateur de travail).

Les résultats seront sauvegardés dans :

- `data_phase2/d_estimates/M1_S0_all_windows_per_k.csv`  
  (une ligne par `(W, G, k)` avec `d_hat_k`, `n_windows`) ;

- `data_phase2/d_estimates/M1_S0_all_windows_summary_core_k.csv`  
  (une ligne par `(W, G)` avec `d_core_mean`, `d_core_std`, `d_core_min`, `d_core_max`).

Ces fichiers fournissent une estimation globale de `d` pour chaque échelle temporelle W,
basée sur **toutes les fenêtres disponibles**, et seront utilisés plus tard pour construire
`T_log(n, d)` avec incertitude.

In [24]:
# Bloc 3.14 – M1 Levina–Bickel sur toutes les fenêtres S0

import numpy as np
import pandas as pd
from pathlib import Path

# 3.14.1 – Vérifier df_ts (S0) et df_windows (définitions de fenêtres)

try:
    df_ts
except NameError:
    raise RuntimeError(
        "df_ts n'est pas défini. Assurez-vous d'avoir exécuté le Bloc 3.0 avant ce bloc."
    )

value_col = "Monthly Mean Total Sunspot Number"

try:
    df_windows
except NameError:
    # recharge depuis window_definitions.csv si nécessaire
    WINDOW_DEFS_PATH = PHASE2_ROOT / "data_phase2" / "windows" / "window_definitions.csv"
    if not WINDOW_DEFS_PATH.exists():
        raise FileNotFoundError(
            f"window_definitions.csv introuvable à : {WINDOW_DEFS_PATH}.\n"
            "Assurez-vous d'avoir exécuté le Bloc 2.4 / 3.0."
        )
    df_windows = pd.read_csv(
        WINDOW_DEFS_PATH,
        parse_dates=["start_date", "end_date"],
    )

print(f"Nombre total de fenêtres S0 : {len(df_windows)}")

# 3.14.2 – Vérifier la présence des fonctions/paramètres M1

try:
    levina_bickel_d_over_k_range
except NameError:
    raise RuntimeError(
        "La fonction levina_bickel_d_over_k_range n'est pas définie. "
        "Assurez-vous d'avoir exécuté le Bloc 3.3 (implémentation M1) avant ce bloc."
    )

try:
    K_MIN, K_MAX, K_CORE_MIN, K_CORE_MAX
except NameError:
    raise RuntimeError(
        "K_MIN/K_MAX ou K_CORE_MIN/K_CORE_MAX ne sont pas définis. "
        "Assurez-vous d'avoir exécuté les Blocs 3.3 et 3.5."
    )

print(f"Paramètres M1 (toutes fenêtres) : K_MIN={K_MIN}, K_MAX={K_MAX}, "
      f"K_CORE_MIN={K_CORE_MIN}, K_CORE_MAX={K_CORE_MAX}")

# 3.14.3 – Application de M1 par combinaison (W, G)

records_per_k_all = []
records_summary_core_all = []

groups = df_windows.groupby(["window_size_months", "stride_months"], sort=True)

for (W, G), group in groups:
    group = group.sort_values("start_index").reset_index(drop=True)
    n_windows = len(group)

    print(f"\nM1 sur toutes les fenêtres S0 pour (W={W}, G={G}) – n_windows={n_windows}")

    # Construction de la matrice X_all (n_windows x W)
    rows = []
    for _, row in group.iterrows():
        start_idx = int(row["start_index"])
        end_idx = int(row["end_index"])

        values = df_ts.iloc[start_idx : end_idx + 1][value_col].to_numpy(dtype=float)
        if len(values) != W:
            raise RuntimeError(
                f"Longueur inattendue pour window_id={row['window_id']} "
                f"(attendu W={W}, obtenu {len(values)})."
            )
        rows.append(values)

    X_all = np.vstack(rows)

    # Calcul de d_hat(k) pour k = K_MIN..K_MAX (borné par n_windows - 1)
    k_max_eff = min(K_MAX, n_windows - 1)
    d_by_k = levina_bickel_d_over_k_range(X_all, K_MIN, k_max_eff)

    # Enregistrement par (W,G,k)
    for k, d_hat_k in d_by_k.items():
        records_per_k_all.append(
            {
                "series": "S0",
                "window_size_months": int(W),
                "stride_months": int(G),
                "k": int(k),
                "d_hat_k": float(d_hat_k),
                "n_windows": int(n_windows),
            }
        )

    # Résumé sur la plage centrale [K_CORE_MIN, K_CORE_MAX]
    d_by_k_core = {
        k: v
        for k, v in d_by_k.items()
        if K_CORE_MIN <= k <= K_CORE_MAX
    }
    if not d_by_k_core:
        raise RuntimeError(
            f"Aucune valeur de k dans la plage centrale [{K_CORE_MIN},{K_CORE_MAX}] "
            f"pour (W={W}, G={G})."
        )

    d_vals_core = np.array(list(d_by_k_core.values()), dtype=float)
    summary_core = {
        "series": "S0",
        "window_size_months": int(W),
        "stride_months": int(G),
        "k_core_min": int(min(d_by_k_core.keys())),
        "k_core_max": int(max(d_by_k_core.keys())),
        "d_core_mean": float(d_vals_core.mean()),
        "d_core_std": float(d_vals_core.std()),
        "d_core_min": float(d_vals_core.min()),
        "d_core_max": float(d_vals_core.max()),
        "n_k_core": int(len(d_vals_core)),
        "n_windows": int(n_windows),
    }
    records_summary_core_all.append(summary_core)

    print(
        f"  -> (W={W}, G={G}) : d_core_mean={summary_core['d_core_mean']:.3f}, "
        f"d_core_std={summary_core['d_core_std']:.3f}, "
        f"d_core_min={summary_core['d_core_min']:.3f}, "
        f"d_core_max={summary_core['d_core_max']:.3f}"
    )

# 3.14.4 – DataFrames et sauvegarde

df_M1_all_per_k = pd.DataFrame.from_records(records_per_k_all)
df_M1_all_summary_core = pd.DataFrame.from_records(records_summary_core_all)

print("\nRésumé global M1 S0 (toutes fenêtres, plage centrale) :")
display(df_M1_all_summary_core)

DEST_DIR = PHASE2_ROOT / "data_phase2" / "d_estimates"
DEST_DIR.mkdir(parents=True, exist_ok=True)

M1_PER_K_ALL_PATH = DEST_DIR / "M1_S0_all_windows_per_k.csv"
M1_SUMMARY_CORE_ALL_PATH = DEST_DIR / "M1_S0_all_windows_summary_core_k.csv"

df_M1_all_per_k.to_csv(M1_PER_K_ALL_PATH, index=False)
df_M1_all_summary_core.to_csv(M1_SUMMARY_CORE_ALL_PATH, index=False)

print("\nFichiers de résultats M1 (S0, toutes fenêtres) sauvegardés :")
print(f"  - {M1_PER_K_ALL_PATH}")
print(f"  - {M1_SUMMARY_CORE_ALL_PATH}")

# 3.14.5 – Logging

log_message(
    "INFO",
    (
        "M1 Levina–Bickel appliqué à S0 sur TOUTES les fenêtres "
        f"pour {len(df_M1_all_summary_core)} combinaisons (W,G). "
        f"Résultats sauvegardés dans {M1_PER_K_ALL_PATH.name} et "
        f"{M1_SUMMARY_CORE_ALL_PATH.name}."
    ),
    block="BLOC_3.14",
)
log_metric(
    "M1_S0_all_windows_combos",
    int(len(df_M1_all_summary_core)),
    extra={
        "per_k_path": str(M1_PER_K_ALL_PATH),
        "summary_core_path": str(M1_SUMMARY_CORE_ALL_PATH),
    },
)

Nombre total de fenêtres S0 : 11682
Paramètres M1 (toutes fenêtres) : K_MIN=5, K_MAX=20, K_CORE_MIN=10, K_CORE_MAX=18

M1 sur toutes les fenêtres S0 pour (W=60, G=1) – n_windows=3206
  -> (W=60, G=1) : d_core_mean=16.575, d_core_std=0.734, d_core_min=15.581, d_core_max=17.935

M1 sur toutes les fenêtres S0 pour (W=60, G=6) – n_windows=535
  -> (W=60, G=6) : d_core_mean=11.170, d_core_std=0.971, d_core_min=9.843, d_core_max=12.963

M1 sur toutes les fenêtres S0 pour (W=60, G=12) – n_windows=268
  -> (W=60, G=12) : d_core_mean=8.418, d_core_std=0.706, d_core_min=7.422, d_core_max=9.591

M1 sur toutes les fenêtres S0 pour (W=132, G=1) – n_windows=3134
  -> (W=132, G=1) : d_core_mean=13.753, d_core_std=0.270, d_core_min=13.511, d_core_max=14.422

M1 sur toutes les fenêtres S0 pour (W=132, G=6) – n_windows=523
  -> (W=132, G=6) : d_core_mean=10.700, d_core_std=0.898, d_core_min=9.500, d_core_max=12.354

M1 sur toutes les fenêtres S0 pour (W=132, G=12) – n_windows=262
  -> (W=132, G=12) : d_

,series,window_size_months,stride_months,k_core_min,k_core_max,d_core_mean,d_core_std,d_core_min,d_core_max,n_k_core,n_windows
0,S0,60,1,10,18,16.574888,0.733524,15.580764,17.935439,9,3206
1,S0,60,6,10,18,11.169843,0.971320,9.843058,12.963494,9,535
2,S0,60,12,10,18,8.417730,0.706307,7.421633,9.590931,9,268
3,S0,132,1,10,18,13.753381,0.269932,13.511196,14.421751,9,3134
4,S0,132,6,10,18,10.700252,0.898186,9.500174,12.354231,9,523
5,S0,132,12,10,18,8.352969,0.552992,7.478232,9.291295,9,262
6,S0,264,1,10,18,9.016448,0.296566,8.662486,9.669331,9,3002
7,S0,264,6,10,18,9.841389,0.634457,9.050302,11.028094,9,501
8,S0,264,12,10,18,9.059071,0.903485,7.840453,10.642914,9,251



Fichiers de résultats M1 (S0, toutes fenêtres) sauvegardés :
  - C:\Users\zackd\OneDrive\Desktop\Phase2_Tlog_v0.5\SunspotPhase2Tlog\data_phase2\d_estimates\M1_S0_all_windows_per_k.csv
  - C:\Users\zackd\OneDrive\Desktop\Phase2_Tlog_v0.5\SunspotPhase2Tlog\data_phase2\d_estimates\M1_S0_all_windows_summary_core_k.csv
[STEP=22][INFO][BLOC_3.14] M1 Levina–Bickel appliqué à S0 sur TOUTES les fenêtres pour 9 combinaisons (W,G). Résultats sauvegardés dans M1_S0_all_windows_per_k.csv et M1_S0_all_windows_summary_core_k.csv.
[METRIC][M1_S0_all_windows_combos] = 9 (step=22)


### Bloc 3.15 – Diagnostic M1 : calibration vs toutes les fenêtres

Les résultats M1 sur **toutes les fenêtres S0** donnent des `d_core_mean` beaucoup plus grands
que ceux obtenus sur le **sous-échantillon de calibration** (par exemple `d ≈ 3.3` en calibration
pour W=60, contre `d ≈ 16.6` sur toutes les fenêtres).

Avant d’aller plus loin, nous comparons systématiquement :

- `M1_S0_calibration_summary_core_k.csv` (sous-échantillon, 40 fenêtres par (W,G))
- `M1_S0_all_windows_summary_core_k.csv` (toutes les fenêtres)

Objectifs de ce bloc :

1. Construire un tableau comparatif avec, pour chaque `(W, G)` :
   - `d_core_mean_calib` vs `d_core_mean_all`
   - différence absolue (`d_diff = all − calib`)
   - rapport (`d_ratio = all / calib`)
2. Visualiser ces écarts pour identifier :
   - si M1 gonfle systématiquement `d` quand on augmente le nombre de fenêtres,
   - si certains couples `(W, G)` sont plus stables que d’autres.

Ces diagnostics décideront ensuite de la *place* de M1 dans le pipeline global
(par exemple : usage limité à des sous‑échantillons, ou comme simple indicateur de tendance).

In [25]:
# Bloc 3.15 – Diagnostic M1 : calibration vs toutes les fenêtres S0

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from pathlib import Path

# 3.15.1 – Chargement des fichiers de résumé M1

CALIB_DIR = PHASE2_ROOT / "data_phase2" / "d_estimates_calibration"
ALL_DIR = PHASE2_ROOT / "data_phase2" / "d_estimates"

CALIB_PATH = CALIB_DIR / "M1_S0_calibration_summary_core_k.csv"
ALL_PATH = ALL_DIR / "M1_S0_all_windows_summary_core_k.csv"

if not CALIB_PATH.exists():
    raise FileNotFoundError(
        f"Fichier de calibration introuvable : {CALIB_PATH}.\n"
        "Assurez-vous d'avoir exécuté les Blocs 3.3, 3.5."
    )

if not ALL_PATH.exists():
    raise FileNotFoundError(
        f"Fichier toutes fenêtres introuvable : {ALL_PATH}.\n"
        "Assurez-vous d'avoir exécuté le Bloc 3.14."
    )

df_calib = pd.read_csv(CALIB_PATH)
df_all = pd.read_csv(ALL_PATH)

print("Aperçu M1 calibration (plage centrale) :")
display(df_calib.head())

print("\nAperçu M1 toutes fenêtres (plage centrale) :")
display(df_all.head())

# 3.15.2 – Fusion sur (series, W, G)

key_cols = ["series", "window_size_months", "stride_months"]

df_merge = df_calib[key_cols + ["d_core_mean", "d_core_std"]].merge(
    df_all[key_cols + ["d_core_mean", "d_core_std", "n_windows"]],
    on=key_cols,
    suffixes=("_calib", "_all"),
)

# 3.15.3 – Calcul des écarts

df_merge["d_diff"] = df_merge["d_core_mean_all"] - df_merge["d_core_mean_calib"]
df_merge["d_ratio"] = df_merge["d_core_mean_all"] / df_merge["d_core_mean_calib"]

print("\nComparaison M1 (calibration vs toutes fenêtres) :")
display(df_merge)

# 3.15.4 – Figures : d_core_mean_calib vs d_core_mean_all

sns.set_style("whitegrid")

# Figure 1 – Scatter calib vs all avec diagonale y=x
plt.figure(figsize=(6, 5))
sns.scatterplot(
    data=df_merge,
    x="d_core_mean_calib",
    y="d_core_mean_all",
    hue="window_size_months",
    style="stride_months",
    s=100,
)
min_val = min(df_merge["d_core_mean_calib"].min(), df_merge["d_core_mean_all"].min())
max_val = max(df_merge["d_core_mean_calib"].max(), df_merge["d_core_mean_all"].max())
plt.plot([min_val, max_val], [min_val, max_val], "k--", label="y = x")

plt.xlabel("d_core_mean (calibration)")
plt.ylabel("d_core_mean (toutes fenêtres)")
plt.title("M1 – d_core_mean calibration vs toutes fenêtres (S0)")
plt.legend()
plt.tight_layout()

ARTIFACTS_DIR = PHASE2_ROOT / "artifacts"
ARTIFACTS_DIR.mkdir(parents=True, exist_ok=True)
FIG1_PATH = ARTIFACTS_DIR / "M1_S0_calib_vs_all_scatter.png"
plt.savefig(FIG1_PATH, dpi=150, bbox_inches="tight")
plt.close()

# Figure 2 – d_ratio en fonction de W, facetté par G
plt.figure(figsize=(7, 4))
sns.barplot(
    data=df_merge,
    x="window_size_months",
    y="d_ratio",
    hue="stride_months",
)
plt.axhline(1.0, color="k", linestyle="--", label="ratio = 1")
plt.ylabel("d_core_mean_all / d_core_mean_calib")
plt.xlabel("window_size_months (W)")
plt.title("M1 – Ratio d_all / d_calib par W et G (S0)")
plt.tight_layout()

FIG2_PATH = ARTIFACTS_DIR / "M1_S0_calib_vs_all_ratio.png"
plt.savefig(FIG2_PATH, dpi=150, bbox_inches="tight")
plt.close()

print("\nFigures de diagnostic M1 sauvegardées :")
print(f"  - {FIG1_PATH}")
print(f"  - {FIG2_PATH}")

# 3.15.5 – Logging

log_message(
    "INFO",
    (
        "Diagnostic M1 (calibration vs toutes fenêtres) généré pour S0. "
        f"Table fusionnée : {len(df_merge)} combinaisons (W,G). "
        f"Figures sauvegardées dans {FIG1_PATH.name} et {FIG2_PATH.name}."
    ),
    block="BLOC_3.15",
)
log_metric(
    "M1_S0_calib_vs_all_diagnostic",
    int(len(df_merge)),
    extra={
        "calib_path": str(CALIB_PATH),
        "all_path": str(ALL_PATH),
        "scatter_figure": str(FIG1_PATH),
        "ratio_figure": str(FIG2_PATH),
    },
)

Aperçu M1 calibration (plage centrale) :


,series,window_size_months,stride_months,k_core_min,k_core_max,d_core_mean,d_core_std,d_core_min,d_core_max,n_k_core
0,S0,60,1,10,18,3.301630,0.404120,2.801035,4.038216,9
1,S0,60,6,10,18,3.413963,0.388647,2.985436,4.125657,9
2,S0,60,12,10,18,3.394886,0.393295,2.920511,4.204442,9
3,S0,132,1,10,18,3.525000,0.209171,3.317934,3.954966,9
4,S0,132,6,10,18,3.603652,0.230090,3.333395,4.045191,9



Aperçu M1 toutes fenêtres (plage centrale) :


,series,window_size_months,stride_months,k_core_min,k_core_max,d_core_mean,d_core_std,d_core_min,d_core_max,n_k_core,n_windows
0,S0,60,1,10,18,16.574888,0.733524,15.580764,17.935439,9,3206
1,S0,60,6,10,18,11.169843,0.971320,9.843058,12.963494,9,535
2,S0,60,12,10,18,8.417730,0.706307,7.421633,9.590931,9,268
3,S0,132,1,10,18,13.753381,0.269932,13.511196,14.421751,9,3134
4,S0,132,6,10,18,10.700252,0.898186,9.500174,12.354231,9,523



Comparaison M1 (calibration vs toutes fenêtres) :


,series,window_size_months,stride_months,d_core_mean_calib,d_core_std_calib,d_core_mean_all,d_core_std_all,n_windows,d_diff,d_ratio
0,S0,60,1,3.301630,0.404120,16.574888,0.733524,3206,13.273258,5.020213
1,S0,60,6,3.413963,0.388647,11.169843,0.971320,535,7.755880,3.271811
2,S0,60,12,3.394886,0.393295,8.417730,0.706307,268,5.022844,2.479532
3,S0,132,1,3.525000,0.209171,13.753381,0.269932,3134,10.228381,3.901669
4,S0,132,6,3.603652,0.230090,10.700252,0.898186,523,7.096600,2.969280
5,S0,132,12,3.563428,0.305607,8.352969,0.552992,262,4.789541,2.344082
6,S0,264,1,4.611742,0.382151,9.016448,0.296566,3002,4.404706,1.955107
7,S0,264,6,4.569296,0.287942,9.841389,0.634457,501,5.272093,2.153809
8,S0,264,12,4.704547,0.350419,9.059071,0.903485,251,4.354524,1.925599



Figures de diagnostic M1 sauvegardées :
  - C:\Users\zackd\OneDrive\Desktop\Phase2_Tlog_v0.5\SunspotPhase2Tlog\artifacts\M1_S0_calib_vs_all_scatter.png
  - C:\Users\zackd\OneDrive\Desktop\Phase2_Tlog_v0.5\SunspotPhase2Tlog\artifacts\M1_S0_calib_vs_all_ratio.png
[STEP=23][INFO][BLOC_3.15] Diagnostic M1 (calibration vs toutes fenêtres) généré pour S0. Table fusionnée : 9 combinaisons (W,G). Figures sauvegardées dans M1_S0_calib_vs_all_scatter.png et M1_S0_calib_vs_all_ratio.png.
[METRIC][M1_S0_calib_vs_all_diagnostic] = 9 (step=23)


### Bloc 3.15b – Synthèse M1_full vs M2 : rôle des méthodes

Les résultats M1 sur **toutes les fenêtres S0** montrent que :

- pour chaque `(W, G)`, `d_core_mean_all` est **2 à 5 fois plus grand** que
  `d_core_mean_calib` (40 fenêtres) ;
- par exemple :
  - W=60, G=1 : `d_calib ≈ 3.30` → `d_all ≈ 16.57`
  - W=132, G=1 : `d_calib ≈ 3.53` → `d_all ≈ 13.75`
  - W=264, G=1 : `d_calib ≈ 4.61` → `d_all ≈ 9.02`

Cela indique que M1 (Levina–Bickel), appliqué à **un grand nombre de fenêtres glissantes**
très corrélées, produit des estimations de `d` fortement **gonflées** et dépendantes de
`n_windows`, en contradiction avec l’idée d’une dimension intrinsèque stable.

Décision méthodologique :

- M1 reste utile comme **méthode de calibration locale** sur des sous‑échantillons de taille
  contrôlée (comme en Blocs 3.3–3.7), et comme outil de comparaison.
- Les estimations M1_full (toutes fenêtres) **ne seront pas utilisées directement** pour les
  conclusions principales ni comme entrée de référence pour `T_log(n, d)`.

En conséquence :

- **M2 (PR/PCA)** devient l’**estimateur de référence** de `d` par `(W, G)` pour la Phase 2
  (complété par M1_calib et M3 comme contrôles).
- Dans la suite, nous appliquons M2 à **toutes les fenêtres S0** pour obtenir des `d_PR` globaux
  robustes par taille de fenêtre W, tout en conservant les diagnostics M1/M3.

### Bloc 3.16 – M2 (PR/PCA) sur *toutes* les fenêtres S0

Nous généralisons maintenant la méthode M2 (Participation Ratio / PCA) à **toutes les
fenêtres S0** définies dans [window_definitions.csv](cci:7://file:///c:/Users/zackd/OneDrive/Desktop/Phase2_Tlog_v0.5/SunspotPhase2Tlog/data_phase2/windows/window_definitions.csv:0:0-0:0).

Pour chaque combinaison `(W, G)` :

1. Nous construisons une matrice `X_all` de taille `(n_windows, W)` :
   - lignes = toutes les fenêtres pour ce `(W, G)` ;
   - colonnes = positions temporelles dans la fenêtre (0..W−1) de la série S0.
2. Nous centrons les colonnes de `X_all`.
3. Nous calculons la matrice de covariance et ses valeurs propres \\( \lambda_i \\).
4. Nous en déduisons :
   - `d_PR` (Participation Ratio dimension) ;
   - `d_PR_80` : nombre minimal de composantes pour ≥80 % de variance ;
   - `d_PR_90` : nombre minimal de composantes pour ≥90 % de variance.

Les résultats sont sauvegardés dans :

- `data_phase2/d_estimates/M2_S0_PR_all_windows_summary.csv`

avec, pour chaque `(W, G)` :

- `d_PR`, `d_PR_80`, `d_PR_90`, `n_windows`, `n_eigvals_pos`.

Ces valeurs serviront comme **estimation principale de `d`** par taille de fenêtre W
pour la Phase 2 et pour la construction de `T_log(n, d)`.